# README

This part of the code is suiting for the bosonic cases.
Please cite [J. Chem. Phys. (in press) (2022)](https://aip.scitation.org/doi/10.1063/5.0095961)


## IMPORT 
Import the library.

### ISSUE
If the numba package is not installed, you can just comment out "from numba import njit"

In [ ]:
import math
from numpy import linalg as LA
from numba import njit

@njit()
def fit_J(w, res, expn, etal, sigma):
    for i in range(len(etal)):
        res += etal[i] / (expn[i] + sigma * 1.j * w)


@njit()
def fit_t(t, res, expn, etal):
    for i in range(len(etal)):
        res += etal[i] * np.exp(-expn[i] * t)
    return res

## Define bath density funciton
### Input

gams1 and lams1 are parameters using in gen_jw.
gen_jw is the detailed density funciton of bath. Ie,

Durde: $\frac{2 \lambda \gamma \omega}{\omega^2 + \gamma^2}$: w * lams1 * gams1 / (w * w + gams1 * gams1)

Ohmic: $\omega \lambda e^{-\omega_{c} |\omega|}$: w * lams1 * np.exp(- gams1 * np.abs(w))

Semicircle: $\lambda \sqrt{\gamma^2 - \omega^2}$: lams1 * np.sqrt(gams1\*\*2 - w\[index\]\*\*2)

### Issue


In [ ]:
gams1 = 5
lams1 = 1

beta = 1

def gen_jw(w):
#     return w * lams1 * np.exp(- gams1 * np.abs(w))
    return w * lams1 * gams1 / (w * w + gams1 * gams1)

#     jw = np.zeros_like(w)
#     index = (w<=gams1) & (w>=-gams1)
#     jw[index] = lams1 * np.sqrt(gams1**2 - w[index]**2)
#     return jw

len_ = 100000
spe_wid = 100
w = np.linspace(-spe_wid, spe_wid, len_)

sigma = 1
jw = gen_jw(w)
jw1 = jw  / (1 - np.exp(- beta * w))
plt.plot(w, jw)
plt.plot(w, jw1)
plt.xlim(-100, 100)

## Preform FFT to obtain c(t)
### Input
n: number of samples in $t$-PFD. The time complexity is around O($n^3$).

scale: length of samples.

n_fft: number of samples in FFT. The time complexity is around O($n\log(n)$).

scale_fft: length of FFT.

### What has been done?
We do the FFT to $c(\omega)$, then sample it to obtain n samples.

In [ ]:
n = 2500
scale = 20

n_fft = 10000000
scale_fft = 100000

n_rate = (scale_fft * scale/ (4 * n))
print(n_rate)
n_rate = int(n_rate)

w = np.linspace(0, scale_fft * np.pi, n_fft + 1)[:-1]
dw = w[1] - w[0]

jw = gen_jw(w)
cw1 = jw / (1 - np.exp(-beta * w))
cw2 = jw / (1 - np.exp(+beta * w))
del jw

cw1[0] = (cw1[1] + cw2[1]) / 4
cw2[0] = (cw1[1] + cw2[1]) / 4
fft_ct = (np.fft.fft(cw1) * dw - np.fft.ifft(cw2) * len(cw2) * dw) / np.pi
fft_t = 2 * np.pi * np.fft.fftfreq(len(cw1), dw)
del cw1, cw2

fft_ct = fft_ct[(scale>=fft_t) & (fft_t >= 0)][::n_rate]
fft_t = fft_t[(scale>=fft_t) & (fft_t >= 0)][::n_rate]

t = fft_t
res_t = np.zeros(len(t), dtype=complex)

## Show the quality of the FFT and the sampling, Part I
If there are two non-coincident curves, you should play with the cell above a little. 

In [ ]:
len_ = 1000000
spe_wid = 10
w = np.linspace(-spe_wid, spe_wid, len_)

sigma = 1
jw = gen_jw(w)
jw1 = jw / (1 - np.exp(-beta * w))
# plt.plot(w, jw)
plt.plot(w, jw1)
fft_ct[0] = fft_ct[0] / 2
plt.plot(2 * np.pi * np.fft.fftfreq(len(fft_ct), fft_t[1] - fft_t[0]),
         len(fft_ct) * (fft_t[1] - fft_t[0]) * np.fft.ifft(fft_ct))
plt.xlim(-10, 10)
fft_ct[0] = fft_ct[0] * 2

fft_ct

## Show the quality of the FFT and the sampling, Part II
The sampled curve should be smooth enough and $c(t) \to 0$ when $t \to \inf$.

In [ ]:
plt.plot(fft_t, np.imag(fft_ct))
plt.plot(fft_t, np.real(fft_ct))
# plt.xlim(0, 10)
plt.show()
print(fft_ct[:10])
print(fft_ct[-10:])

## PFD main cell
This cell do the PFD scheme to obtain $\eta_{k}$ and $\gamma_{k}$.

n\_gamma should be large enough to obtain a good result, and it means that vs\[n\_gamma\] should be small enough.

In [ ]:
n_sample = n + 1
h = np.imag(fft_ct)
H = np.zeros((n_sample, n_sample))
for i in range(n_sample):
    H[i, :] = h[i:n_sample + i]
sing_vs, Q = LA.eigh(H)
# del H
phase_mat = np.diag(
    [np.exp(-1j * np.angle(sing_v) / 2.0) for sing_v in sing_vs])
vs = np.array([np.abs(sing_v) for sing_v in sing_vs])
Qp = np.dot(Q, phase_mat)
sort_array = np.argsort(vs)[::-1]
vs = vs[sort_array]
Qp = (Qp[:, sort_array])
vs = vs[:20]
Qp = Qp[:, :20]
print(vs)

for n_gamma in [1]:
    print("len of gamma", n_gamma)
    gamma = np.roots(Qp[:, n_gamma][::-1])
    gamma_new = gamma[np.argsort(np.abs(gamma))[:n_gamma]]
    t_imag = 2 * n * np.log(gamma_new)
    gamma_m = np.zeros((n_sample * 2 - 1, n_gamma), dtype=complex)
    for i in range(n_gamma):
        for j in range(n_sample * 2 - 1):
            gamma_m[j, i] = gamma_new[i]**j
    omega_imag = np.dot(LA.inv(np.dot(np.transpose(gamma_m), gamma_m)),
                        np.dot(np.transpose(gamma_m), np.transpose(h)))

    res_t = np.zeros(len(t), dtype=complex)
    fit_t(fft_t, res_t, -t_imag / scale, omega_imag)
    plt.plot(fft_t, np.imag(fft_ct) - res_t)
    plt.savefig("imag_{}.pdf".format(n_gamma))
    plt.clf()

h = np.real(fft_ct)
H = np.zeros((n_sample, n_sample))
for i in range(n_sample):
    H[i, :] = h[i:n_sample + i]
sing_vs, Q = LA.eigh(H)
# del H
phase_mat = np.diag(
    [np.exp(-1j * np.angle(sing_v) / 2.0) for sing_v in sing_vs])
vs = np.array([np.abs(sing_v) for sing_v in sing_vs])
Qp = np.dot(Q, phase_mat)
sort_array = np.argsort(vs)[::-1]
vs = vs[sort_array]
Qp = (Qp[:, sort_array])
vs = vs[:20]
Qp = Qp[:, :20]
print(vs)

for n_gamma in [2]:
    print("len of gamma", n_gamma)
    gamma = np.roots(Qp[:, n_gamma][::-1])
    gamma_new = gamma[np.argsort(np.abs(gamma))[:n_gamma]]
    t_real = 2 * n * np.log(gamma_new)
    gamma_m = np.zeros((n_sample * 2 - 1, n_gamma), dtype=complex)
    for i in range(n_gamma):
        for j in range(n_sample * 2 - 1):
            gamma_m[j, i] = gamma_new[i]**j
    omega_real = np.dot(LA.inv(np.dot(np.transpose(gamma_m), gamma_m)),
                        np.dot(np.transpose(gamma_m), np.transpose(h)))

    res_t = np.zeros(len(t), dtype=complex)
    fit_t(fft_t, res_t, -t_real / scale, omega_real)
    plt.plot(fft_t, np.real(fft_ct) - res_t)
    plt.savefig("real_{}.pdf".format(n_gamma))
    plt.clf()

## Validation of PFD
Show the error of PFD in the frequency domain. In partice, you can omit some small $\eta$

In [ ]:
etal1 = np.append(1.j * omega_imag, omega_real)
etar1 = np.append(np.conjugate(1.j * omega_imag), np.conjugate(omega_real))
etaa1 = np.append(np.abs(omega_imag), np.abs(omega_real))
expn1 = np.append(-t_imag / scale, -t_real / scale)

# filter_ = 1e-3
# expn1 = expn1[np.abs(etal1) > filter_]
# etal1 = etal1[np.abs(etal1) > filter_]

len_ = 100000
spe_wid = 50
w = np.linspace(-spe_wid, spe_wid, len_)
phixx = gen_jw(w) / (1 - np.exp(- beta * w))
res_J1 = np.zeros(len(w), dtype=complex)
fit_J(w, res_J1, expn1, etal1, -1)
plt.plot(w, (phixx.real))
plt.plot(w, (res_J1.real), "--")
plt.xlim(-5, 15)
plt.savefig("res_ohmic.pdf")
plt.show()

len_ = 100000
spe_wid = 20
w = np.linspace(-spe_wid, spe_wid, len_)
phixx = gen_jw(w) / (1 - np.exp(- beta * w))

res_J1 = np.zeros(len(w), dtype=complex)
fit_J(w, res_J1, expn1, etal1, -1)
plt.plot(w, (phixx.real-res_J1.real), "--")

## Gen input file 

In [ ]:
expn_imag_sort = np.argsort(np.abs(np.imag(expn1)))[::-1]
expn_imag = np.sort(np.abs(np.imag(expn1)))[::-1]
expn1 = expn1[expn_imag_sort]
etal1 = etal1[expn_imag_sort]
etar1 = etal1[expn_imag_sort]
expn_val_cc = np.where(expn1[expn_imag > 1e-10])[0]
etaa1 = np.zeros(len(etal1), dtype=float)
for ii in range(0, len(expn_val_cc), 2):
    even_i = ii
    odd_i = ii + 1
    etar1[even_i] = np.conj(etal1[odd_i])
    etar1[odd_i] = np.conj(etal1[even_i])
    etaa1[even_i] = np.abs(etal1[even_i])
    etaa1[odd_i] = np.abs(etal1[odd_i])
for ii in range(len(expn_val_cc), len(expn1)):
    even_i = ii
    etar1[even_i] = np.conj(etal1[even_i])
    etaa1[even_i] = np.abs(etal1[even_i])

nmod = 1
expn = expn1.copy()
etal = etal1.copy()
etar = etar1.copy()
etaa = etaa1.copy()

In [ ]:
def arma_write(ndarray, filename):

    shape = ndarray.shape
    dimen = len(shape)

    with open(filename, 'w') as f:
        if dimen == 1:
            if issubclass(type(ndarray[0]), np.int_):
                print('ARMA_MAT_TXT_IS004\n%d %d' % (shape[0], 1), file=f)
                for row in ndarray:
                    print('%d' % row, file=f)
            elif issubclass(type(ndarray[0]), float):
                print('ARMA_MAT_TXT_FN008\n%d %d' % (shape[0], 1), file=f)
                for row in ndarray:
                    print('%.8e' % row, file=f)
            elif issubclass(type(ndarray[0]), complex):
                print('ARMA_MAT_TXT_FC016\n%d %d' % (shape[0], 1), file=f)
                for row in ndarray:
                    print('(%.8e,%-.8e)' % (row.real, row.imag), file=f)

        elif dimen == 2:

            if issubclass(type(ndarray[0, 0]), np.int_):
                print('ARMA_MAT_TXT_IS004\n%d %d' % (shape[0], shape[1]),
                      file=f)
                for row in ndarray:
                    print(' '.join('%d' % x for x in row), file=f)
            elif issubclass(type(ndarray[0, 0]), float):
                print('ARMA_MAT_TXT_FN008\n%d %d' % (shape[0], shape[1]),
                      file=f)
                for row in ndarray:
                    print(' '.join('%.8e' % x for x in row), file=f)
            elif issubclass(type(ndarray[0, 0]), complex):
                print('ARMA_MAT_TXT_FC016\n%d %d' % (shape[0], shape[1]),
                      file=f)
                for row in ndarray:
                    print(' '.join('(%.8e,%-.8e)' % (x.real, x.imag)
                                   for x in row),
                          file=f)

        elif dimen == 3:

            if issubclass(type(ndarray[0, 0, 0]), np.int_):
                print('ARMA_CUB_TXT_IS004\n%d %d %d' %
                      (shape[1], shape[2], shape[0]),
                      file=f)
                for slc in ndarray:
                    for row in slc:
                        print(' '.join('%d' % x for x in row))
            elif issubclass(type(ndarray[0, 0, 0]), float):
                print('ARMA_CUB_TXT_FN008\n%d %d %d' %
                      (shape[1], shape[2], shape[0]),
                      file=f)
                for slc in ndarray:
                    for row in slc:
                        print(' '.join('%-.8e' % x for x in row), file=f)
            elif issubclass(type(ndarray[0, 0, 0]), complex):
                print('ARMA_CUB_TXT_FC016\n%d %d %d' %
                      (shape[1], shape[2], shape[0]),
                      file=f)
                for slc in ndarray:
                    for row in slc:
                        print(' '.join('(%.8e,%-.8e)' % (x.real, x.imag)
                                       for x in row),
                              file=f)


arma_write(expn, 'inp_expn.mat')
arma_write(etal, 'inp_etal.mat')
arma_write(etar, 'inp_etar.mat')
arma_write(etaa, 'inp_etaa.mat')

## 